In [1]:
# Dependencies
import requests
import json
from pprint import pprint
import pandas as pd
from pathlib import Path

# Import the API key
from config import geoapify_key

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
all_zip_data = pd.read_csv(
    Path('zip_codes.csv')   
)

# Review the DataFrame
all_zip_data.head()

,postcode,latitude,longitude
0,3000-vic,-37.814214,144.963145
1,3001-vic,-37.814327,144.962604
2,3002-vic,-37.812872,144.981886
3,3003-vic,-37.809332,144.925223
4,3004-vic,-37.837281,144.972710


In [3]:
#so because we put the params we dont need to put them into the base URL
# Set the search parameters
countrycode = "au"
params = { 
    "apiKey": geoapify_key,
    "filter": countrycode
}

# Build URL using the geocode endpoint
base_url = "https://api.geoapify.com/v2/places"

# Create a list to store the results
results = []

In [4]:
# base URL
base_url = "https://api.geoapify.com/v1/places"

#parameters for type of establishment 
categories = "healthcare.clinic_or_praxis.general"
bias = f"proximity:{longitude},{latitude}"
limit = 3

# Create an empty DataFrame to store the results
closest_doc = pd.DataFrame(columns=["postcode", "address_line1", "address_line2"])

# Iterate through each postcode in the range from 3000 to 3008 (inclusive)
for postcode in range(3000, 3009):
    # Look up the corresponding latitude and longitude from the all_zip_data DataFrame
    row = all_zip_data[all_zip_data["postcode"] == f"{postcode}-vic"]
    
    if not row.empty:
        latitude = row.iloc[0]["latitude"]
        longitude = row.iloc[0]["longitude"]

    # Make the API request to find the closest doctors
    params = {
        "categories":categories,
        "lat": latitude,
        "lon": longitude,
        "limit":limit,
        "bias":bias,
        "apiKey":geoapify_key 
    }
    

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()

        if data.get("features"):
            # Sort the doctor locations by distance
            doctor_locations = sorted(data["features"], key=lambda x: x["properties"]["distance"])
            
            # Take the top 3 closest doctors (you can adjust the number as needed)
            closest_doctors = doctor_locations[:3]

            for doctor in closest_doctors:
                address_line1 = doctor["properties"]["address_line1"]
                address_line2 = doctor["properties"]["address_line2"]
                
                # Append the data to the result DataFrame
                result_df = result_df.append({
                    "postcode": postcode,
                    "address_line1": address_line1,
                    "address_line2": address_line2
                }, ignore_index=True)
        else:
            print(f"No doctor results found for postcode: {postcode}")
    else:
        print(f"Request failed for postcode: {postcode} with status code: {response.status_code}")

# Print or save the result DataFrame
print(closest_doc)

# If you want to save the results to a CSV file:
# result_df.to_csv("closest_doctors.csv", index=False)


NameError: name 'longitude' is not defined

In [5]:
# Set the geographical coordinates for Brisbane, Australia
latitude = -37.814214 
longitude = 144.963145

# Set the parameters for the type of place
categories = "healthcare.clinic_or_praxis.general"

# Set the parameters for the type of search
bias = f"proximity:{longitude},{latitude}"
limit = 3

# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [6]:
# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
pprint(response)

{'features': [{'geometry': {'coordinates': [144.966178, -37.81360020053079],
                            'type': 'Point'},
               'properties': {'address_line1': 'Swanston Street Medical Centre',
                              'address_line2': '267 Bourke Street, Melbourne '
                                               'VIC 3000, Australia',
                              'categories': ['healthcare',
                                             'healthcare.clinic_or_praxis',
                                             'healthcare.clinic_or_praxis.general'],
                              'city': 'Melbourne',
                              'country': 'Australia',
                              'country_code': 'au',
                              'datasource': {'attribution': '© OpenStreetMap '
                                                            'contributors',
                                             'license': 'Open Database Licence',
                                  

In [11]:
# Define a function to find doctors
def find_doctors(latitude, longitude):
    categories = "healthcare.clinic_or_praxis.general"
    bias = f"proximity:{longitude},{latitude}"
    limit = 3

    params = {
        "categories": categories,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        doctors = []

        for place in data['features']:
            doctor_name = place['properties']['name']
            doctor_address = place['properties']['formatted']

            doctors.append({
                'Name': doctor_name,
                'Address': doctor_address
            })

        return doctors
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.content)
        return []

# Loop through the DataFrame and find doctors for each location
doctors_data = []

for index, row in all_zip_data.iloc[:10].iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    doctors = find_doctors(latitude, longitude)
    
    doctors_data.append({
        'postcode': row['postcode'],
        'doctors': doctors
    })

# Now, doctors_data contains the top 3 doctors for each location
# You can process and save this data as needed

doctors_399 = pd.DataFrame(doctors_data)
doctors_399.head()

,postcode,doctors
0,3000-vic,"[{'Name': 'Swanston Street Medical Centre', 'A..."
1,3001-vic,"[{'Name': 'Swanston Street Medical Centre', 'A..."
2,3002-vic,"[{'Name': 'No 68 Medical Centre', 'Address': '..."
3,3003-vic,"[{'Name': 'Kensington Medical Clinic', 'Addres..."
4,3004-vic,"[{'Name': 'Albert Park Medical Centre', 'Addre..."


In [17]:
# Define a function to find doctors
def find_doctor(latitude, longitude):
    categories = "healthcare.clinic_or_praxis.general"
    bias = f"proximity:{longitude},{latitude}"
    limit = 3

    params = {
        "categories": categories,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        doctors = []

        for place in data['features']:
            doctor_name = place['properties']['name']
            doctor_address = place['properties']['formatted']

            doctors.append({
                'Name': doctor_name,
                'Address': doctor_address
            })

        return doctors
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.content)
        return []

# Create an empty DataFrame to store the data
doctor_399_df = pd.DataFrame(columns=['postcode', 'Name', 'Address'])

# Loop through the first 10 rows of the DataFrame and find doctors for each location
for index, row in all_zip_data.iloc[:5].iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    doctors = find_doctors(latitude, longitude)
    
    for doctor in doctors:
        doctor_399_df = doctor_399_df.append({
            'postcode': row['postcode'],
            'Name': doctor['Name'],
            'Address': doctor['Address']
        }, ignore_index=True)

doctor_399_df.head()

C:\Users\tamik\AppData\Local\Temp\ipykernel_11344\680937016.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  doctor_399_df = doctor_399_df.append({
C:\Users\tamik\AppData\Local\Temp\ipykernel_11344\680937016.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  doctor_399_df = doctor_399_df.append({
C:\Users\tamik\AppData\Local\Temp\ipykernel_11344\680937016.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  doctor_399_df = doctor_399_df.append({
C:\Users\tamik\AppData\Local\Temp\ipykernel_11344\680937016.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  doctor_399_df = doctor_399_df.append({
C:\Users\tamik\AppData\Local\Temp\ip

,postcode,Name,Address
0,3000-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
1,3000-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."
2,3000-vic,Royal Park Medical,"Royal Park Medical, Flemington Road, North Mel..."
3,3001-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
4,3001-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."


In [32]:
# Define a function to find doctors
def find_doctor(latitude, longitude):
    categories = "healthcare.clinic_or_praxis.general"
    bias = f"proximity:{longitude},{latitude}"
    limit = 3

    params = {
        "categories": categories,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        doctors = []

        for place in data['features']:
            doctor_name = place['properties'].get('name', 'Unknown Doctor')  # Use a default value if 'name' is missing
            doctor_address = place['properties'].get('formatted', 'Unknown Address')  # Use a default value if 'formatted' is missing

            doctors.append({
                'Name': doctor_name,
                'Address': doctor_address
            })

        return doctors
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.content)
        return []

# Create an empty DataFrame to store the data
doctor_99_df = pd.DataFrame(columns=['postcode', 'Name', 'Address'])
data_to_concat = []

# Loop through the first 10 rows of the DataFrame and find doctors for each location
for index, row in all_zip_data.iloc[:99].iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    doctors = find_doctor(latitude, longitude)
    
    for doctor in doctors:
        data_to_concat.append({
            'postcode': row['postcode'],
            'Name': doctor['Name'],
            'Address': doctor['Address']
        })

doctor_99_df = pd.concat([doctor_99_df, pd.DataFrame(data_to_concat)], ignore_index=True)
doctor_99_df

,postcode,Name,Address
0,3000-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
1,3000-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."
2,3000-vic,Royal Park Medical,"Royal Park Medical, Flemington Road, North Mel..."
3,3001-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
4,3001-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."
...,...,...,...
292,3097-vic,Polaris Medical Centre,"Polaris Medical Centre, Copernicus Crescent, B..."
293,3097-vic,Sexual health Victoria,"Sexual health Victoria, 901 Whitehorse Road, B..."
294,3098-vic,Centro de Saúde Mehara,"Centro de Saúde Mehara, Road Mehara-Tutuala, P..."
295,3098-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."


In [22]:
# Define a function to find doctors
def find_doctor(latitude, longitude):
    categories = "healthcare.clinic_or_praxis.general"
    bias = f"proximity:{longitude},{latitude}"
    limit = 3

    params = {
        "categories": categories,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        doctors = []

        for place in data['features']:
            doctor_name = place['properties']['name']
            doctor_address = place['properties']['formatted']

            doctors.append({
                'Name': doctor_name,
                'Address': doctor_address
            })

        return doctors
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.content)
        return []

# Create an empty DataFrame to store the data
doctor_299_df = pd.DataFrame(columns=['postcode', 'Name', 'Address'])
data_to_concat = []

# Loop through the first 10 rows of the DataFrame and find doctors for each location
for index, row in all_zip_data.iloc[100:299].iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    doctors = find_doctor(latitude, longitude)
    
    for doctor in doctors:
        data_to_concat.append({
            'postcode': row['postcode'],
            'Name': doctor['Name'],
            'Address': doctor['Address']
        })

doctor_299_df = pd.concat([doctor_299_df, pd.DataFrame(data_to_concat)], ignore_index=True)
doctor_299_df

,postcode,Name,Address
0,3100-vic,Centro de Saúde Mehara,"Centro de Saúde Mehara, Road Mehara-Tutuala, P..."
1,3100-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
2,3100-vic,HR Maubisse,"HR Maubisse, Estrada Aileu - Ainaro, Maubisse,..."
3,3101-vic,Dentists of Hawthorn,"Dentists of Hawthorn, 330 Burwood Road, Hawtho..."
4,3101-vic,Wingrove Medical Clinic,"Wingrove Medical Clinic, 270 Wingrove Street, ..."
...,...,...,...
592,3297-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
593,3297-vic,HR Maubisse,"HR Maubisse, Estrada Aileu - Ainaro, Maubisse,..."
594,3298-vic,Seaford Central Medical Clinic,"Seaford Central Medical Clinic, 134 Nepean Hig..."
595,3298-vic,The Smile Place Mt Eliza,"The Smile Place Mt Eliza, 1299 Nepean Highway,..."


In [25]:
# Define a function to find doctors
def find_doctor(latitude, longitude):
    categories = "healthcare.clinic_or_praxis.general"
    bias = f"proximity:{longitude},{latitude}"
    limit = 3

    params = {
        "categories": categories,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        doctors = []

        for place in data['features']:
            doctor_name = place['properties'].get('name', 'Unknown Doctor')  # Use a default value if 'name' is missing
            doctor_address = place['properties'].get('formatted', 'Unknown Address')  # Use a default value if 'formatted' is missing

            doctors.append({
                'Name': doctor_name,
                'Address': doctor_address
            })

        return doctors
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.content)
        return []

# Create an empty DataFrame to store the data
doctor_399_df = pd.DataFrame(columns=['postcode', 'Name', 'Address'])
data_to_concat = []

# Loop through the first 10 rows of the DataFrame and find doctors for each location
for index, row in all_zip_data.iloc[300:399].iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    doctors = find_doctor(latitude, longitude)
    
    for doctor in doctors:
        data_to_concat.append({
            'postcode': row['postcode'],
            'Name': doctor['Name'],
            'Address': doctor['Address']
        })

doctor_399_df = pd.concat([doctor_399_df, pd.DataFrame(data_to_concat)], ignore_index=True)
doctor_399_df


,postcode,Name,Address
0,3300-vic,Ferrers Medical Clinic,"Ferrers Medical Clinic, Wehl Street North, Mou..."
1,3300-vic,Newtown Medical Centre,"Newtown Medical Centre, 311 Pakington Street, ..."
2,3300-vic,Forest Road Medical Centre,"Forest Road Medical Centre, 3 Forest Road Sout..."
3,3301-vic,Ferrers Medical Clinic,"Ferrers Medical Clinic, Wehl Street North, Mou..."
4,3301-vic,Newtown Medical Centre,"Newtown Medical Centre, 311 Pakington Street, ..."
...,...,...,...
292,3398-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
293,3398-vic,HR Maubisse,"HR Maubisse, Estrada Aileu - Ainaro, Maubisse,..."
294,3399-vic,Centro de Saúde Mehara,"Centro de Saúde Mehara, Road Mehara-Tutuala, P..."
295,3399-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."


In [26]:
# Define a function to find doctors
def find_doctor(latitude, longitude):
    categories = "healthcare.clinic_or_praxis.general"
    bias = f"proximity:{longitude},{latitude}"
    limit = 3

    params = {
        "categories": categories,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        doctors = []

        for place in data['features']:
            doctor_name = place['properties'].get('name', 'Unknown Doctor')  # Use a default value if 'name' is missing
            doctor_address = place['properties'].get('formatted', 'Unknown Address')  # Use a default value if 'formatted' is missing

            doctors.append({
                'Name': doctor_name,
                'Address': doctor_address
            })

        return doctors
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.content)
        return []

# Create an empty DataFrame to store the data
doctor_599_df = pd.DataFrame(columns=['postcode', 'Name', 'Address'])
data_to_concat = []

# Loop through the first 10 rows of the DataFrame and find doctors for each location
for index, row in all_zip_data.iloc[400:599].iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    doctors = find_doctor(latitude, longitude)
    
    for doctor in doctors:
        data_to_concat.append({
            'postcode': row['postcode'],
            'Name': doctor['Name'],
            'Address': doctor['Address']
        })

doctor_599_df = pd.concat([doctor_599_df, pd.DataFrame(data_to_concat)], ignore_index=True)
doctor_599_df

,postcode,Name,Address
0,3401-vic,Ferrers Medical Clinic,"Ferrers Medical Clinic, Wehl Street North, Mou..."
1,3401-vic,Unknown Doctor,Unknown Address
2,3401-vic,Forest Road Medical Centre,"Forest Road Medical Centre, 3 Forest Road Sout..."
3,3402-vic,Centro de Saúde Mehara,"Centro de Saúde Mehara, Road Mehara-Tutuala, P..."
4,3402-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
...,...,...,...
592,3598-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
593,3598-vic,HR Maubisse,"HR Maubisse, Estrada Aileu - Ainaro, Maubisse,..."
594,3599-vic,Unknown Doctor,Unknown Address
595,3599-vic,Deakin Medical Centre,"Deakin Medical Centre, 255 Deakin Avenue, Mild..."


In [27]:
# Define a function to find doctors
def find_doctor(latitude, longitude):
    categories = "healthcare.clinic_or_praxis.general"
    bias = f"proximity:{longitude},{latitude}"
    limit = 3

    params = {
        "categories": categories,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        doctors = []

        for place in data['features']:
            doctor_name = place['properties'].get('name', 'Unknown Doctor')  # Use a default value if 'name' is missing
            doctor_address = place['properties'].get('formatted', 'Unknown Address')  # Use a default value if 'formatted' is missing

            doctors.append({
                'Name': doctor_name,
                'Address': doctor_address
            })

        return doctors
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.content)
        return []

# Create an empty DataFrame to store the data
doctor_799_df = pd.DataFrame(columns=['postcode', 'Name', 'Address'])
data_to_concat = []

# Loop through the first 10 rows of the DataFrame and find doctors for each location
for index, row in all_zip_data.iloc[600:799].iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    doctors = find_doctor(latitude, longitude)
    
    for doctor in doctors:
        data_to_concat.append({
            'postcode': row['postcode'],
            'Name': doctor['Name'],
            'Address': doctor['Address']
        })

doctor_799_df = pd.concat([doctor_799_df, pd.DataFrame(data_to_concat)], ignore_index=True)
doctor_799_df

,postcode,Name,Address
0,3601-vic,Centro de Saúde Mehara,"Centro de Saúde Mehara, Road Mehara-Tutuala, P..."
1,3601-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
2,3601-vic,HR Maubisse,"HR Maubisse, Estrada Aileu - Ainaro, Maubisse,..."
3,3602-vic,Centro de Saúde Mehara,"Centro de Saúde Mehara, Road Mehara-Tutuala, P..."
4,3602-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
...,...,...,...
592,3799-vic,Mooroolbark Superclinic,"Mooroolbark Superclinic, 108 Brice Avenue, Moo..."
593,3799-vic,Gladstone Street Medical Center,"Gladstone Street Medical Center, 46 Gladstone ..."
594,3800-vic,Family Medical Practice,"Family Medical Practice, 84 Springvale Road, S..."
595,3800-vic,Medical Centre Dr DC John Ho,"Medical Centre Dr DC John Ho, 16 Balmoral Aven..."


In [28]:
# Define a function to find doctors
def find_doctor(latitude, longitude):
    categories = "healthcare.clinic_or_praxis.general"
    bias = f"proximity:{longitude},{latitude}"
    limit = 3

    params = {
        "categories": categories,
        "limit": limit,
        "bias": bias,
        "apiKey": geoapify_key
    }

    base_url = "https://api.geoapify.com/v2/places"
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        doctors = []

        for place in data['features']:
            doctor_name = place['properties'].get('name', 'Unknown Doctor')  # Use a default value if 'name' is missing
            doctor_address = place['properties'].get('formatted', 'Unknown Address')  # Use a default value if 'formatted' is missing

            doctors.append({
                'Name': doctor_name,
                'Address': doctor_address
            })

        return doctors
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.content)
        return []

# Create an empty DataFrame to store the data
doctor_999_df = pd.DataFrame(columns=['postcode', 'Name', 'Address'])
data_to_concat = []

# Loop through the first 10 rows of the DataFrame and find doctors for each location
for index, row in all_zip_data.iloc[800:999].iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    doctors = find_doctor(latitude, longitude)
    
    for doctor in doctors:
        data_to_concat.append({
            'postcode': row['postcode'],
            'Name': doctor['Name'],
            'Address': doctor['Address']
        })

doctor_999_df = pd.concat([doctor_999_df, pd.DataFrame(data_to_concat)], ignore_index=True)
doctor_999_df

,postcode,Name,Address
0,3802-vic,Better Health Family Clinic,"Better Health Family Clinic, 127-129 Somervill..."
1,3802-vic,Somerville Road Medical Centre,"Somerville Road Medical Centre, 133 Somerville..."
2,3802-vic,Family Medical Practice,"Family Medical Practice, 84 Springvale Road, S..."
3,3803-vic,Better Health Family Clinic,"Better Health Family Clinic, 127-129 Somervill..."
4,3803-vic,Somerville Road Medical Centre,"Somerville Road Medical Centre, 133 Somerville..."
...,...,...,...
586,3997-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
587,3997-vic,HR Maubisse,"HR Maubisse, Estrada Aileu - Ainaro, Maubisse,..."
588,3998-vic,Centro de Saúde Mehara,"Centro de Saúde Mehara, Road Mehara-Tutuala, P..."
589,3998-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."


In [34]:
#dataframes to concatinate 
dataframes_to_combine = [doctor_99_df, doctor_299_df, doctor_399_df, 
                         doctor_599_df, doctor_799_df, doctor_999_df]

#concatinate all dataframes 
all_doctors_data = pd.concat(dataframes_to_combine, ignore_index=True)

all_doctors_data

,postcode,Name,Address
0,3000-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
1,3000-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."
2,3000-vic,Royal Park Medical,"Royal Park Medical, Flemington Road, North Mel..."
3,3001-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
4,3001-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."
...,...,...,...
2971,3997-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."
2972,3997-vic,HR Maubisse,"HR Maubisse, Estrada Aileu - Ainaro, Maubisse,..."
2973,3998-vic,Centro de Saúde Mehara,"Centro de Saúde Mehara, Road Mehara-Tutuala, P..."
2974,3998-vic,CENTRO SAUDE DE LAISOROLAI,"CENTRO SAUDE DE LAISOROLAI, Dalan Kiik ba Aber..."


In [35]:
#Export to csv file 
output_file_path= Path('Resources/all_doctors_data.csv') 
all_doctors_data.to_csv(output_file_path, index=False)

In [37]:
# When a postcode does not exist geoapify gave it the longitude and lattitude 
#of -12.4643506	130.842693 so the same 3 doctors in spain so i want to remove 
#these rows from the dataframe 
spain_doctors = ["CENTRO SAUDE DE LAISOROLAI",
                "HR Maubisse", "Centro de Saúde Mehara"]

invalid_postcode_rows = all_doctors_data[all_doctors_data["Name"].isin(spain_doctors)]

cleaned_doctors_df = all_doctors_data.drop(invalid_postcode_rows.index)
cleaned_doctors_df

,postcode,Name,Address
0,3000-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
1,3000-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."
2,3000-vic,Royal Park Medical,"Royal Park Medical, Flemington Road, North Mel..."
3,3001-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
4,3001-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."
...,...,...,...
2965,3995-vic,Rise Medical Centre,"Rise Medical Centre, Fairwood Rise, Officer VI..."
2966,3995-vic,The Smile Place Mt Eliza,"The Smile Place Mt Eliza, 1299 Nepean Highway,..."
2967,3996-vic,Gladstone Street Medical Center,"Gladstone Street Medical Center, 46 Gladstone ..."
2968,3996-vic,Rise Medical Centre,"Rise Medical Centre, Fairwood Rise, Officer VI..."


In [38]:
#Export to csv file 
output_file_path= Path('Resources/cleaned_doctors_data.csv') 
cleaned_doctors_df.to_csv(output_file_path, index=False)

In [39]:
#create a file with the index as a column to use as a primary key when 
#making a database 
doctor_address_data = cleaned_doctors_df.reset_index()
doctor_address_data.head()

,index,postcode,Name,Address
0,0,3000-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
1,1,3000-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."
2,2,3000-vic,Royal Park Medical,"Royal Park Medical, Flemington Road, North Mel..."
3,3,3001-vic,Swanston Street Medical Centre,"Swanston Street Medical Centre, 267 Bourke Str..."
4,4,3001-vic,No 68 Medical Centre,"No 68 Medical Centre, 68 Lonsdale Street, Melb..."


In [40]:
#Export to csv file 
output_file_path= Path('Resources/doctor_address_data.csv') 
doctor_address_data.to_csv(output_file_path, index=False)

In [43]:
cleaned_doctors_df.to_json('Resources/doctors_data.json')